# 11. iterator를 병렬로 처리하려면 zip을 사용하자

In [1]:
'''
리스트 컴프리헨션 사용하여 source list -> derived list 얻기 (참고: 7장 map, filter 대신 리스트 컴프리헨션)
'''
names = ['Cecilia', 'Lise', 'Marie'] #source list: 이름 (str) 저장
letters = [len(n) for n in names] #derived list: names에 있는 이름들의 길이(int) 저장

In [2]:
'''
source list & derived list: 서로의 인덱스로 연관되어 있음
만약에 두 리스트를 병렬로(함께) 순회하고 싶다면 source list의 길이만큼 loop를 돌면 됨.
'''
longest_name = None #가장 긴 이름
max_letters = 0 #가장 긴 이름 길이

for i in range(len(names)):
    count = letters[i]
    if count > max_letters:
        longest_name = names[i]
        max_letters = count

print(longest_name)

Cecilia


In [3]:
'''
But, names & letters 리스트에 인덱스로 접근하는 방법 --> 코드 읽기 어려움, 인덱스 i로 배열에 접근하는 동작이 두 번 일어남
enumerate를 사용하면 약간 개선 가능
'''
longest_name = None #가장 긴 이름
max_letters = 0 #가장 긴 이름 길이

for i, name in enumerate(names): #enumerate를 사용하면 index와 element 모두 접근 가능
    count = letters[i]
    if count > max_letters:
        longest_name = name
        max_letters = count

print(longest_name)

Cecilia


In [4]:
'''
내장 함수 zip: 지연 제너레이터, 이터레이터 두 개 이상 감싼다.
- 각 이터레이터로부터 다음 값을 담은 튜플을 가져옴.
- 다중 리스트에서 인덱스로 접근하는 코드보다 훨씬 명료함.

[iterator, generator 참고]
https://mingrammer.com/translation-iterators-vs-generators/
- iterator는 현재 내부 상황을 기억하고 next() 호출 시 다음 원소를 넘겨주는 헬퍼 객체임.
- 모든 generator는 iterator이며, >>요청 시에 하나의 값만 생성<<하는 게으른(idle) 작동을 함.
'''
longest_name = None #가장 긴 이름
max_letters = 0 #가장 긴 이름 길이

for name, count in zip(names, letters):
    if count > max_letters:
        longest_name = name
        max_letters = count
        
print(longest_name)

Cecilia


In [5]:
'''
[내장 함수 zip 사용 시 문제점]

1. python2 제공 zip은 제너레이터가 아니다.
- 제공한 이터레이터를 완전히 순회해서 zip으로 생성한 모든 튜플을 반환함.
- 이 과정에서 메모리를 많이 사용하여 프로그램이 망가지는 원인이 될 수 있음.
- 따라서 매우 큰 이터레이터를 zip으로 묶어서 사용하고 싶다면, 내장 모듈 itertools에 있는 izip 사용해야 함. (16장 참고)

2. 입력 이터레이터들 길이가 다르면 zip이 이상하게 동작한다.
- zip은 감싼 이터레이터가 끝날 때까지 튜플을 넘겨주는데, 실행할 리스트 길이가 다르면 zip의 잘라내기 동작이 이상하고 나쁘게 작동함.
- 만약 리스트 길이가 같다고 확신할 수 없다면, 내장 모듈 itertools의 zip_longest (python2 izip_longest) 사용 고려.
'''
names.append('Rosalind')
for name, count in zip(names, letters):
    print(name) #위에서 추가한 'Rosalind' 출력되지 않음.

Cecilia
Lise
Marie


In [6]:
from itertools import zip_longest

for name, count in zip_longest(names, letters):
    print(name) #zip_longest 사용하면 Rosalind까지 출력됨.

Cecilia
Lise
Marie
Rosalind


**핵심 정리**
- 내장 함수 zip은 여러 이터레이터를 병렬로 순회할 때 사용할 수 있다.
- python3의 zip은 튜플을 생성하는 지연 제너레이터다. 한편, python2의 zip은 전체 결과를 튜플 리스트로 반환한다.
- 길이가 다른 이터레이터를 사용하면 zip은 그 결과를 조용히 잘라낸다.
- 내장 모듈 itertools의 zip_longest 함수를 사용하면, 여러 이터레이터를 길이에 상관없이 병렬로 순회할 수 있다.

# 12. for, while loop 뒤에는 else 블록을 쓰지 말자

In [7]:
'''
파이썬 루프문에서 반복되는 내부 블록 바로 다음에 else 블록을 둘 수 있음.
그러나 기대와는 다르게(?) else 블록은 루프가 종료되지마자 실행되는 것을 확인할 수 있음.
'''
for i in range(3):
    print('Loop %d' % i)
else:
    print('Else block!')

Loop 0
Loop 1
Loop 2
Else block!


- if/else 문: 이전 블록이 실행되지 않으면 else 블록을 실행
- try/except 문: 이전 블록에서 실패하면 except 블록을 실행
- try/except/else 문: 이전 블록에서 실패하지 않으면 else 블록을 실행
- try/finally 문: 이전 블록을 실행하고 항상 마지막에 finally 블록을 실행
- for/else 문: 루프가 완료되지 않으면 else 블록을 실행? >>> 반대입니다!

In [8]:
'''
for loop에서 break를 사용해야 else 블록을 건너뛸 수 있음
'''
for i in range(3):
    print('Loop %d' % i)
    if i == 1:
        break
else:
    print('Else block!')

Loop 0
Loop 1


In [10]:
'''
빈 시퀀스를 처리하는 루프문에서도 else 블록이 즉시 실행됨
'''
for x in []:
    print('Never runs')
else:
    print('For Else block!')

For Else block!


In [11]:
'''
while loop가 처음부터 거짓인 경우에도 else 블록 실행됨
'''
while False:
    print('Never runs')
else:
    print('While Else block!')

While Else block!


왜 이렇게 동작할까 ? ? ?
- 루프 다음에 오는 else 블록은 뭔가를 검색할 때 유용함.
- 예컨데, 두 숫자가 서로소인지 판별하는 문제에서 모든 옵션을 루프 내에서 테스트하고, 
만약 break를 만나 중단되지 않았다면 else 블록에서 두 숫자가 서로소일 때 실행되도록 할 수 있음.

In [12]:
'''
위의 서로소 예시. 그런데 이런 방식으로 코드를 작성하지 말라는 게 이 챕터 결론임.
>>> 대신에 이런 계산을 수행하는 헬퍼 함수를 작성합시다!
'''
a = 4
b = 9
for i in range(2, min(a,b)+1):
    print('Testing', i)
    if a % i == 0 and b % i == 0: #두 숫자가 공약수를 가질 때
        print('Not coprime')
        break
else:
    print('Coprime')

Testing 2
Testing 3
Testing 4
Coprime


In [13]:
'''
1) 원하는 조건을 찾았을 때, 바로 return 하기. 
만약 루프가 실패로 끝난다면 기본 결과 (True) return 하면 됨.
'''
def coprime(a, b):
    for i in range(2, min(a,b)+1):
        if a % i == 0 and b % i == 0:
            return False
    return True

coprime(4, 9) #서로소임 (True)

True

In [14]:
'''
2) 루프에서 찾으려는 대상을 찾았는지 알려주는 결과 변수를 사용하기. 
뭔가를 찾았다면, 즉시 break로 루프를 중단한다.
'''
def coprime2(a, b):
    is_coprime = True
    for i in range(2, min(a,b)+1):
        if a % i ==0 and b % i == 0:
            is_coprime = False
            break
    return is_coprime

coprime2(2,4) #서로소가 아님 (False)

False

핵심정리
- 파이썬에는 for, while loop 내부 블록 바로 뒤에 else 블록을 사용할 수 있게 하는 특별한 문법이 있음.
- 루프 본문이 break 문을 만나지 않은 경우에만 루프 다음에 오는 else 블록이 실행됨.
- 루프 뒤에 else 블록을 사용하면 직관적이지 않고 혼동하기 쉬우니 사용하지 말자. (결론)

# 13. try/except/else/finally에서 각 블록의 장점을 이용하자

**finally 블록**
- 예외를 전달하고 싶지만, 예외가 발생해도 정리 코드를 실행하고 싶을 때 사용 (e.g 파일 핸들러 종료)
- 즉, finally 블록은 try 블록 테스트 후에 항상 실행되는 것이 보장됨.  

In [15]:
handle = open('something_data.txt') #IOError 발생 (파일이 없는 경우)
try:
    data = handle.read()
finally:
    handle.close()

FileNotFoundError: [Errno 2] No such file or directory: 'something_data.txt'

In [17]:
'''
[try]: read 메서드에서 발생한 예외는 항상 호출 코드까지 전달됨
[finally]: close 메서드는 실행되는 것이 보장됨
'''
handle = open('text_data.txt') #IOError 발생 가능 (그러나 try 블록에서 캐치하지 않음)
try:
    data = handle.read() #UnicodeDecodeError 발생 가능
finally:
    handle.close() #try 블록 이후 항상 실행

In [18]:
handle.closed #return True if file is closed

True

**else 블록**
- 코드에서 어떤 예외를 처리하고 처리하고, 어떤 예외를 전달할지 명확하게 하고 싶을 때 사용
- try 블록에서 예외가 발생하지 않으면, else 블록이 실행됨.
- else 블록을 사용하면 try 블록의 코드를 최소로 줄이고 가독성을 높일 수 있음 (잡고 싶은 예외가 있는 코드만 작성하면 되니까.)

In [19]:
import json

def load_json_key(data, key):
    try:
        result_dict = json.loads(data) #ValueError가 일어날 수 있음
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key] #KeyError가 일어날 수 있음

In [20]:
'''
올바른 JSON 파일이 아닌 경우 --> loads 함수에서 디코드할 때 ValueError 발생 가능
'''
with open('text_data.txt','r') as fp:
    text_data = fp.read()
load_json_key(text_data, 'some_key') #JSONDecodeError 발생하는 거 확인 가능

KeyError: 

In [21]:
'''
올바른 JSON 파일이지만 key가 없는 경우 --> else 블록에서 keyerror 발생함
'''
with open('json_data.json','r') as fp:
    json_data = fp.read()
load_json_key(json_data, 'key2')

KeyError: 'key2'

In [22]:
'''
올바른 JSON 파일이고 key가 있는 경우 --> 문제없이 else 블록 실행
'''
load_json_key(json_data, 'key1')

'value1'

**모두 함께 사용하기**
- 복합문 하나로 모든 것을 처리하고 싶을 때, try/except/else/finally 사용

In [23]:
'''
[try]: 파일을 읽고 처리 (여기서는 분자/분모 계산)
[except]: try 블록에서 발생하는 예외 처리 (여기서는 0이 분모가 되는 예외)
[else]: 파일을 즉석에서 업데이트하고 (result 저장) 이와 관련한 예외가 전달되게 하는 데 사용
[finally]: 파일 핸들을 정리하는 데 사용함. else 블록에서 예외가 발생하더라도 항상 실행되어 파일 핸들을 닫음
'''

UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+') #IOError 발생 가능
    try:
        data = handle.read()
        op = json.loads(data)
        value = (
            op['numerator'] / 
            op['denominator']
        )
    except ZeroDivisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handle.write(result)
        return value
    finally:
        handle.close()

In [28]:
with open('zero_division_data.json','r') as fp:
    json_data = json.load(fp)

json_data #denominator == 0

{'denominator': 0, 'numerator': 3}

In [29]:
UNDEFINED

In [30]:
divide_json('zero_division_data.json')

In [31]:
with open('division_data.json','r') as fp:
    json_data = json.load(fp)

json_data

{'denominator': 6, 'numerator': 3}

In [32]:
divide_json('division_data.json')

0.5

In [33]:
with open('division_data.json','r') as fp:
    json_data = json.load(fp)

json_data #result 항목이 updata 된 것 확인 가능

{'denominator': 6, 'numerator': 3, 'result': 0.5}

핵심 정리
- try/finally 문: try 블록에서 예외 발생 여부와 상관없이 정리 코드 실행 가능
- else 블록: try 블록 코드 양을 최소로 줄이는 데 도움을 줌. try/except 블록과 성공한 경우 (즉 예외가 발생하지 않은 경우) 실행할 코드를 시각적으로 구분해줌
- else 블록: try 블록이 성공적으로 실행된 후 finally 블록에서 공통 정리 코드를 실행하기 전에 추가 작업을 하는 데 사용할 수 있음